In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import json
import numpy as np
from scripts.load_data import *

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor

import matplotlib.pyplot as plt
from pandas import read_csv
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [4]:
# data_student_numbers_f = pd.read_excel("../data/input/student_count_first-years.xlsx")
# data_student_numbers_h = pd.read_excel("../data/input/student_count_higher-years.xlsx")
# data_student_numbers_v = pd.read_excel("../data/input/student_volume.xlsx")

configuration = load_configuration("../configuration/configuration.json")

paths = configuration["paths"]
data_student_numbers_f = pd.read_excel(paths["path_student_count_first-years"])
data_student_numbers_h = pd.read_excel(paths["path_student_count_higher-years"])
data_student_numbers_v = pd.read_excel(paths["path_student_volume"])

In [5]:
combined_data_student_numbers = data_student_numbers_f.merge(data_student_numbers_h, on=["Croho groepeernaam", "Herkomst", "Collegejaar"])
combined_data_student_numbers = combined_data_student_numbers.rename(columns={"Aantal_studenten_x": "Aantal_studenten_f", "Aantal_studenten_y": "Aantal_studenten_h"})
combined_data_student_numbers

,Collegejaar,Croho groepeernaam,Herkomst,Aantal_studenten_f,Aantal_studenten_h
0,2012,M Geneeskunde,NL,180,868
1,2013,M Geneeskunde,NL,173,892
2,2014,M Geneeskunde,NL,175,916
3,2015,M Geneeskunde,NL,21,1086
4,2016,M Geneeskunde,NL,148,984
...,...,...,...,...,...
2176,2016,M Leraar Voorbereidend Hoger Onderwijs in Gods...,NL,1,1
2177,2014,M Leraar Voorbereidend Hoger Onderwijs in Natu...,NL,3,1
2178,2015,M Leraar Voorbereidend Hoger Onderwijs in Natu...,NL,1,1
2179,2017,M Leraar Voorbereidend Hoger Onderwijs in Natu...,NL,2,1


In [6]:
combined_data_student_numbers[combined_data_student_numbers["Croho groepeernaam"] == "B Sociologie"]

,Collegejaar,Croho groepeernaam,Herkomst,Aantal_studenten_f,Aantal_studenten_h
1839,2012,B Sociologie,NL,49,76
1840,2013,B Sociologie,NL,51,88
1841,2014,B Sociologie,NL,38,108
1842,2015,B Sociologie,NL,32,105
1843,2016,B Sociologie,NL,44,95
1844,2017,B Sociologie,NL,31,98
1845,2018,B Sociologie,NL,43,89
1846,2019,B Sociologie,NL,48,99
1847,2020,B Sociologie,NL,63,99
1848,2021,B Sociologie,NL,86,111


In [7]:
predict_year = 2023
programme = "B Sociologie"
origin = "NL"

In [8]:
data = combined_data_student_numbers
data = data.sort_values(by=["Collegejaar"])
train = data[(data["Collegejaar"] < predict_year) & (data['Croho groepeernaam'] == programme) & (data["Herkomst"] == origin)]
# test = data[(data["Collegejaar"] == predict_year - 1) & (data['Croho groepeernaam'] == programme) & (data["Herkomst"] == origin)]

train = train.drop_duplicates()

In [9]:
train = train["Aantal_studenten_h"].values
# test = test["Aantal_studenten_h"].values

In [10]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [17]:
look_back = 3
nr_of_epochs = 10

In [18]:
train_dataset = []
for d in train:
    train_dataset.append([d])
print(train_dataset)

scaler = MinMaxScaler(feature_range=(0, 1))
train_dataset = scaler.fit_transform(train_dataset)

X_train, Y_train = create_dataset(train_dataset, look_back)

trainX = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))

model = Sequential()
model.add(LSTM(input_shape=(1, look_back), units=4))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, Y_train, epochs=nr_of_epochs, batch_size=1, verbose=2)

[[76], [88], [108], [105], [95], [98], [89], [99], [99], [111], [141]]
Epoch 1/10


c:\Users\jjble\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 - 2s - 259ms/step - loss: 0.1560
Epoch 2/10
7/7 - 0s - 4ms/step - loss: 0.1440
Epoch 3/10
7/7 - 0s - 10ms/step - loss: 0.1325
Epoch 4/10
7/7 - 0s - 5ms/step - loss: 0.1230
Epoch 5/10
7/7 - 0s - 7ms/step - loss: 0.1128
Epoch 6/10
7/7 - 0s - 8ms/step - loss: 0.1038
Epoch 7/10
7/7 - 0s - 6ms/step - loss: 0.0951
Epoch 8/10
7/7 - 0s - 7ms/step - loss: 0.0867
Epoch 9/10
7/7 - 0s - 6ms/step - loss: 0.0798
Epoch 10/10
7/7 - 0s - 5ms/step - loss: 0.0724


In [19]:
scaler.inverse_transform(np.reshape(train_dataset, (len(train_dataset), 1)))

array([[ 76.],
       [ 88.],
       [108.],
       [105.],
       [ 95.],
       [ 98.],
       [ 89.],
       [ 99.],
       [ 99.],
       [111.],
       [141.]])

In [22]:
test = train_dataset[-look_back:]
print(test)

reshaped_test = np.reshape(test, (1, 1, look_back))
print(scaler.inverse_transform(np.reshape(test, (1, look_back))))

prediction = model.predict(reshaped_test, verbose=2)
prediction = scaler.inverse_transform(prediction)
print(f"Prediction: {prediction}")

[[0.35384615]
 [0.53846154]
 [1.        ]]
[[ 99. 111. 141.]]
1/1 - 0s - 306ms/step
Prediction: [[87.21108]]
